In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, regularizers
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler

In [3]:
#Setup dataset

track_data = pd.read_csv("./data/data.csv")
segment_data = pd.read_csv("./data/timbre.csv")
data = track_data.append(segment_data)

labels = data['order']
# data['key'] = data['key']/11
# data['loudness'] = data['loudness']/-32
# data['tempo'] = data['tempo']/250
# data['time_signature'] = data['time_signature']/5

columns_to_remove = ['Unnamed: 0', 'id', 'duration_ms', 'track_title', 'album_title', 'album_artist', 'track_number', 'total_tracks']#, 'order']
data.drop(columns = columns_to_remove, axis = 1, inplace = True)

data.loc[:,['key', 'loudness', 'tempo', 'time_signature']] = MinMaxScaler().fit_transform(data.loc[:,['key', 'loudness', 'tempo', 'time_signature']])

training_set, test_set, training_labels, test_labels = train_test_split(data, labels, test_size=0.01, random_state=42)

data
# training_set

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,order,song_timbre,song_timbre_start,song_timbre_end,loudness_start,loudness_end
0,0.788,0.616,0.818182,0.679207,0.0,0.0337,0.00286,0.549,0.0952,0.6960,0.479872,0.8,0.076923,NaN,NaN,NaN,NaN,NaN
1,0.712,0.500,0.818182,0.638903,0.0,0.0338,0.82500,0.314,0.1060,0.6820,0.362973,0.8,0.153846,NaN,NaN,NaN,NaN,NaN
2,0.740,0.682,0.818182,0.687129,0.0,0.0504,0.06920,0.530,0.3810,0.4100,0.456345,0.8,0.230769,NaN,NaN,NaN,NaN,NaN
3,0.702,0.297,0.909091,0.570477,0.0,0.0317,0.89400,0.485,0.1030,0.0676,0.443909,0.8,0.307692,NaN,NaN,NaN,NaN,NaN
4,0.775,0.585,0.909091,0.715661,0.0,0.0271,0.04220,0.619,0.0770,0.5180,0.443207,0.8,0.384615,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.344807,0.668408,0.332985,-12.079868,-27.131633
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288666,0.520910,0.312240,-16.789944,-39.224625
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.317270,0.545912,0.348919,-18.519230,-33.865739
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.421664,0.568079,0.497520,-28.177122,-24.270614


In [ ]:
# Old autoencoder

latent_dim = 1

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(12, activation='sigmoid'),
      #layers.Reshape((12))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  
autoencoder = Autoencoder(latent_dim)

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [120]:
# Setup autoencoder

input_track = tf.keras.Input(shape=(13,))

# encoded = layers.Dense(13, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_track)

encoded = layers.Dense(10, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)
encoded = layers.Dense(10, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)
# encoded = layers.Dense(7, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)
# encoded = layers.Dense(7, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)
# encoded = layers.Dense(5, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)
# encoded = layers.Dense(5, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)
encoded = layers.Dense(3, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)
encoded = layers.Dense(3, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)
# encoded = layers.Dense(2, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)
# encoded = layers.Dense(2, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(input_track)

# decoded = layers.Dense(3, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)
# decoded = layers.Dense(3, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)
# decoded = layers.Dense(5, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)
# decoded = layers.Dense(5, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)
decoded = layers.Dense(7, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)
decoded = layers.Dense(7, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)
# decoded = layers.Dense(10, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)
# decoded = layers.Dense(10, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)
decoded = layers.Dense(13, activation='sigmoid', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)
decoded = layers.Dense(13, activation='sigmoid', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))(encoded)

autoencoder = tf.keras.Model(input_track, decoded)

encoder = tf.keras.Model(input_track, encoded)

encoded_input = tf.keras.Input(shape=(3,))
decoder_layer = autoencoder.layers[-1]
decoder = tf.keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss=losses.BinaryCrossentropy())

In [121]:
autoencoder.fit(training_set, training_set,
                epochs=200,
                shuffle=True,
                validation_split = 0.1)

Epoch 1/200
139/139 [==============================] - ETA: 0s - loss: 0.685 - 0s 2ms/step - loss: 0.6828 - val_loss: 0.6714
Epoch 2/200
139/139 [==============================] - 0s 1ms/step - loss: 0.6638 - val_loss: 0.6523
Epoch 3/200
139/139 [==============================] - 0s 1ms/step - loss: 0.6392 - val_loss: 0.6175
Epoch 4/200
139/139 [==============================] - 0s 1ms/step - loss: 0.6091 - val_loss: 0.5923
Epoch 5/200
139/139 [==============================] - 0s 1ms/step - loss: 0.5932 - val_loss: 0.5823
Epoch 6/200
139/139 [==============================] - 0s 1ms/step - loss: 0.5866 - val_loss: 0.5778
Epoch 7/200
139/139 [==============================] - 0s 1ms/step - loss: 0.5831 - val_loss: 0.5751
Epoch 8/200
139/139 [==============================] - 0s 997us/step - loss: 0.5808 - val_loss: 0.5731
Epoch 9/200
139/139 [==============================] - 0s 1ms/step - loss: 0.5789 - val_loss: 0.5714
Epoch 10/200
139/139 [==============================] - 0s 1ms/st

In [122]:
encoded_tracks = encoder.predict(test_set)
decoded_tracks = decoder.predict(encoded_tracks)

# print(encoded_tracks[0])
# print(decoded_tracks[0])
# print(test_set.iloc[0])
# for i in range(13):
#     print(decoded_tracks[0][i]-test_set.iloc[0, i])

distances = []
for i in range(len(decoded_tracks)):
    distances.append(np.linalg.norm(decoded_tracks[i]-test_set.iloc[i]))
np.mean(distances)

0.6759738844438687